In [8]:
import numpy as np
import pandas as pd

In [9]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # to make jupyter print all outputs, not just the last one
from IPython.core.display import HTML # to pretty print pandas df and be able to copy them over (e.g. to ppt slides)

In [10]:
matrix_df = pd.read_parquet('cleaned/netflix_parquet')

In [11]:
matrix_df

,movieId,year,title,review_data
0,1,2003,Dinosaur Planet,"{'0': '{'userId': '1488844', 'rating': 3.0, 'd..."
1,2,2004,Isle of Man TT 2004 Review,"{'0': '{'userId': '2059652', 'rating': 4.0, 'd..."
2,3,1997,Character,"{'0': '{'userId': '1025579', 'rating': 4.0, 'd..."
3,4,1994,Paula Abdul's Get Up & Dance,"{'0': '{'userId': '1065039', 'rating': 3.0, 'd..."
4,5,2004,The Rise and Fall of ECW,"{'0': '{'userId': '1745265', 'rating': 5.0, 'd..."
...,...,...,...,...
768,13508,1999,The League of Gentlemen: Series 1,"{'0': '{'userId': '577397', 'rating': 2.0, 'da..."
769,13509,1998,Little City,"{'0': '{'userId': '998229', 'rating': 3.0, 'da..."
770,13510,1959,Last Train from Gun Hill,"{'0': '{'userId': '779760', 'rating': 3.0, 'da..."
771,13511,1993,Much Ado About Nothing,"{'0': '{'userId': '1392773', 'rating': 4.0, 'd..."


### Feature engineering:

A user-item matrix will be created.

In [18]:
matrix_df = matrix_df.drop(['year','title'],axis=1)
print(matrix_df)

     movieId                                        review_data
0          1  {'0': '{'userId': '1488844', 'rating': 3.0, 'd...
1          2  {'0': '{'userId': '2059652', 'rating': 4.0, 'd...
2          3  {'0': '{'userId': '1025579', 'rating': 4.0, 'd...
3          4  {'0': '{'userId': '1065039', 'rating': 3.0, 'd...
4          5  {'0': '{'userId': '1745265', 'rating': 5.0, 'd...
..       ...                                                ...
768    13508  {'0': '{'userId': '577397', 'rating': 2.0, 'da...
769    13509  {'0': '{'userId': '998229', 'rating': 3.0, 'da...
770    13510  {'0': '{'userId': '779760', 'rating': 3.0, 'da...
771    13511  {'0': '{'userId': '1392773', 'rating': 4.0, 'd...
772    13512  {'0': '{'userId': '1392773', 'rating': 4.0, 'd...

[773 rows x 2 columns]


#### Let's work with movies and reviews first, add other features later:

Only first two keys of dictionary will be kept to accomplish this.

In [19]:
# matrix_df['review_data'] = matrix_df['review_data'].apply(lambda x: None if x is None else [{'userId': review['userId'], 'rating': review['rating']} for review in x.values()])

First we need to find unique user and item(movie)ids:

In [20]:
# # set is used, because it does not allow for duplicates
# user_ids = set()  

# # iterate over each row
# for index, row in df.iterrows():
#     # iterate over each dictionary in the 'review_data' column of the current row
#     for key, value in row['review_data'].items():
#         user_id = value['userId']
#         user_ids.add(user_id)  # Add user ID to the set

# user_ids = list(user_ids)

In [21]:
# # put movieids in set so duplicates are not allowed for here either
# item_ids = list(set(matrix_df['movieId'].unique()))

We will initialize the matrix now:

In [22]:
# num_users = len(user_ids)
# num_items = len(item_ids)
# user_item_matrix = np.zeros((num_items, num_users))

Now we will populate the matrix with the matrix_df values:

In [23]:
# # this for loop looks into each values of both keys in the dictionary in the review_data column
# for i, reviews in enumerate(matrix_df['review_data']):
#     for review in reviews:
#         user_idx = user_ids.index(review['userId'])
#         user_item_matrix[i, user_idx] = review['rating']